In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


Preprocessing

In [2]:
data = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv",encoding='latin-1')

data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data = data.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)


In [4]:
data.columns = ["label", "txt"]
data.head()

,label,txt
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data.shape

(5572, 2)

In [7]:
data['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

Remove Punctuatuions

In [8]:
import string
punc = string.punctuation

In [9]:
 def remove_punc(text):
     return "".join([c for c in text if c not in punc])

s = "thi is text. * ?l"
print(remove_punc(s))

thi is text  l


In [10]:
data["no punc"] = data['txt'].apply(lambda x: remove_punc(x))

In [11]:
data.head()

,label,txt,no punc
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...


**Tokenization**

In [12]:
import re
def tokenize(text):
    return re.split("\W+", text)

data["tokenized"] = data["no punc"].apply(lambda x: tokenize(x.lower()))
data.head()

,label,txt,no punc,tokenized
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l..."


**Stopword Removal**

In [13]:
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [14]:
def remove_stopwords(text):
    return [token for token in text if token not in sw]


tokens = ["I", "me", "stopwordds", "the"]
print(remove_stopwords(tokens))

['I', 'stopwordds']


In [15]:
data["no_stop"] = data["tokenized"].apply(lambda x: remove_stopwords(x))
data.head()

,label,txt,no punc,tokenized,no_stop
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t..."


In [21]:
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test =train_test_split(X_features, data["labels"])

NameError: name 'X_features' is not defined

In [20]:
from sklearn.metrics import precision_score, recall_score


Word to Vec

In [1]:
import gensim

In [14]:
import gensim.downloader as api
api.info().keys()


dict_keys(['corpora', 'models'])

In [15]:

api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [22]:
wiki_embedding = api.load('glove-wiki-gigaword-100')

In [20]:
wiki_embedding.similar_by_word('hello')

[('goodbye', 0.7905024290084839),
 ('hey', 0.7171452641487122),
 ('!', 0.6594691276550293),
 ('yeah', 0.6267022490501404),
 ('dear', 0.6220601201057434),
 ('mister', 0.6092501282691956),
 ('wow', 0.6079446077346802),
 ('muddah', 0.6048929691314697),
 ('mama', 0.6034210920333862),
 ('thank', 0.599325954914093)]

In [56]:
data['text_clean']= data['txt'].apply(lambda x: gensim.utils.simple_preprocess(x))
data.head()

,label,txt,text_clean
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup..."
3,ham,U dun say so early hor... U c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, don, think, he, goes, to, usf, he, lives..."


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text_clean'], data['label'], test_size = 0.2)

In [27]:
wv_model = gensim.models.Word2Vec(X_train, vector_size = 100, window = 5, min_count = 2)

In [31]:
wv_model.wv.similar_by_word('hello')

[('my', 0.9994995594024658),
 ('he', 0.9994860291481018),
 ('them', 0.9994829297065735),
 ('they', 0.9994753003120422),
 ('already', 0.9994741082191467),
 ('im', 0.9994643926620483),
 ('the', 0.9994533061981201),
 ('well', 0.9994513988494873),
 ('off', 0.9994505643844604),
 ('ask', 0.9994495511054993)]

**RNN**

In [5]:
from sklearn.model_selection import train_test_split
labels = np.where(data['label']=='spam', 1, 0)
X_train, X_test, y_train, y_test = train_test_split(data['txt'], labels, test_size =0.2)

In [25]:
labels[0]

0

In [27]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [28]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.6/620.6 MB 48.5 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 94.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.5 MB/s  0:00:00
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard━━━━━━━━━━━━━ 0/4 [ml_dtypes]
    Found existing installation: tensorboard 2.18.032m0/4 [ml_dtypes]
    Uninstalling tensorboard-2.18.0:━━━━━━━━ 0/4 [ml_dtypes]
      Successfully uninstalled tensorboard-2.18.0━━━━━━━━━━━━━━━━━ 1/4 [tensorboard]
  Attempting uninstall: keras━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [tensorboard]
    Found existing installation: keras 3.8.0━━━━━━━━━━━━━━━━━━ 1/4 [tensorboard]
    Uninstalling keras-3.8.0:╺━━━━━━━━━━━━━━━━━━━ 2/4 [keras]d]
      Successfully u

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [30]:

X_train, X_test, y_train, y_test = train_test_split(data['txt'], labels, test_size = 0.2)

In [31]:
max_words = 10000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)

In [32]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

برای این که بتونیم دنباله هارو توی مدل استفاده کنیم نیاز به استفاده از بچ هست
برای استفاده از بچ باید طول رشته ها برابر باشه 
پدینگی که پایین استفاده می کنیم طول همه رشته ها رو برابر با طول طولانی ترین رشته می کنه و باقی رو صفر می ذاره

In [33]:
X_train_seq_padded = pad_sequences(X_train_seq)
X_test_seq_padded = pad_sequences(X_test_seq)
X_train_seq_padded[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [34]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim = max_words, output_dim = 50))
model.add(LSTM(32, dropout = 0, recurrent_dropout =0))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [35]:
model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy', Precision(), Recall()])

In [36]:
history = model.fit(X_train_seq_padded, y_train, 
                   batch_size = 32, epochs = 10,
                   validation_data = (X_test_seq_padded, y_test))

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - accuracy: 0.8617 - loss: 0.3887 - precision_3: 0.5515 - recall_3: 0.2247 - val_accuracy: 0.9830 - val_loss: 0.0538 - val_precision_3: 0.9833 - val_recall_3: 0.8741
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.9911 - loss: 0.0374 - precision_3: 0.9875 - recall_3: 0.9468 - val_accuracy: 0.9910 - val_loss: 0.0299 - val_precision_3: 0.9630 - val_recall_3: 0.9630
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.9978 - loss: 0.0109 - precision_3: 0.9990 - recall_3: 0.9852 - val_accuracy: 0.9901 - val_loss: 0.0359 - val_precision_3: 0.9921 - val_recall_3: 0.9259
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.9995 - loss: 0.0027 - precision_3: 0.9984 - recall_3: 0.9980 - val_accuracy: 0.9937 - val_loss: 0.0253 - val_precision_3: 0.9848 - val_recall_3: 0.9630
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.9996 - loss: 0.0028 - precision_3: 1.0000 - recall_3: 0.

# **RNN Word2Vec**# 

In [57]:
import gensim

tokenizer =  Tokenizer(num_words = max_words)
clean_X_train = X_train.apply(lambda x: gensim.utils.simple_preprocess(x))
wv_model = gensim.models.Word2Vec(clean_X_train, vector_size = 100, window = 5, min_count = 2)

In [62]:
max_words = 10000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
embedding_size = 100

In [63]:
embedding_matrix = np.zeros((max_words, embedding_size))

In [65]:

for word, index in tokenizer.word_index.items():
    if index < max_words:
        if word in wv_model.wv:
            embedding_matrix[index] = wv_model.wv[word]

In [67]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim = max_words, output_dim = embedding_size,
                    weights =[embedding_matrix],
                   trainable = False))
model.add(LSTM(32, dropout = 0, recurrent_dropout =0))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [68]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy', Precision(), Recall()]
             )

In [69]:
history = model.fit(X_train_seq_padded, y_train, 
                   batch_size = 32, epochs = 10,
                   validation_data = (X_test_seq_padded, y_test))

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - accuracy: 0.8604 - loss: 0.4291 - precision_4: 0.4077 - recall_4: 0.0625 - val_accuracy: 0.8942 - val_loss: 0.2686 - val_precision_4: 0.6889 - val_recall_4: 0.2296
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.8862 - loss: 0.2758 - precision_4: 0.7009 - recall_4: 0.3075 - val_accuracy: 0.8960 - val_loss: 0.2599 - val_precision_4: 0.7568 - val_recall_4: 0.2074
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - accuracy: 0.8873 - loss: 0.2721 - precision_4: 0.6943 - recall_4: 0.4154 - val_accuracy: 0.8996 - val_loss: 0.2591 - val_precision_4: 0.6667 - val_recall_4: 0.3407
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.8870 - loss: 0.2631 - precision_4: 0.6991 - recall_4: 0.3919 - val_accuracy: 0.9031 - val_loss: 0.2291 - val_precision_4: 0.7077 - val_recall_4: 0.3407
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.9018 - loss: 0.2320 - precision_4: 0.6875 - recall_4: 0.

In [ ]:
import gensim.downloader as api
wv_model = api.load('glove-wiki-gigaword-100')

In [72]:
embedding_size = 100
embedding_matrix = np.zeros((max_words, embedding_size))
for word, index in tokenizer.word_index.items():
    if index < max_words:
        if word in wv_model.wv:
            embedding_matrix[index] = wv_model[word]

TypeError: 'Word2Vec' object is not subscriptable